# Model Selection Lab

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold, RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer
pd.set_option('display.max_columns', 500)
seed=0

## 1. Preliminaries (from previous lab)

In [12]:
# Read in subset of footballer data
model_data = pd.read_csv('footballer_reduced.csv')

# Turn category into numeric variables
model_data = pd.get_dummies(model_data, drop_first=True)

# Define our X and y
y = model_data.overall
X = model_data.drop('overall', axis = 'columns')

# Split into train&validation, test
# Random state assures that folds are consistent across models
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 50, random_state = 0)

print(Xtrain.shape, Xtest.shape)
model_data.head()

(310, 5) (50, 5)


,age,height_cm,weight_kg,overall,work_rate_att_Low,work_rate_att_Medium
0,20,175,70,58,0,1
1,29,183,80,65,0,0
2,35,183,78,67,0,0
3,24,178,72,69,0,1
4,23,173,73,70,0,1


## 2. Use cross-validation to select the best model 

In [13]:
# Model 1
model1 = LinearRegression()
model1 = model1.fit(Xtrain, ytrain)
trainloss = mean_squared_error(ytrain, model1.predict(Xtrain))
print(f"Training loss: %.3f" % trainloss)

kf = KFold(n_splits=5, shuffle=False)
# kf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=seed)

sc = make_scorer(mean_squared_error)
cv_scores = cross_val_score(model1, Xtrain, ytrain, cv=kf, scoring=sc)
print(f"CV loss:  %.3f +/- %.3f" % (cv_scores.mean(), cv_scores.std()))

Training loss: 34.648
CV loss:  36.009 +/- 3.330


In [14]:
# Model 2: Squared trend for age
# Construct a new feature - age squared
Xtrain2 = Xtrain
Xtrain2 = Xtrain2.assign(age2 = Xtrain.age**2)
model2 = LinearRegression()
model2 = model2.fit(Xtrain2, ytrain)
trainloss = mean_squared_error(ytrain, model2.predict(Xtrain2))
print(f"Training loss: %.3f" % trainloss)

# Print CV scores and include standard deviation. Is it an unbiased estimate?
cv_scores = cross_val_score(model2, Xtrain2, ytrain, cv=kf, scoring=sc)
print(f"CV loss: %.3f +/- %.3f" % (cv_scores.mean(), cv_scores.std()))

Training loss: 31.955
CV loss: 33.409 +/- 4.420


In [15]:
# Model 3: All polynomial features
PT =  PolynomialFeatures(degree=2) #, include_bias=True)
Xtrain3 = PT.fit_transform(Xtrain)
print('Xtrain3 is of size ', Xtrain3.shape)

# Run the linear regression
model3 = LinearRegression().fit(Xtrain3,ytrain)
trainloss = mean_squared_error(ytrain,model3.predict(Xtrain3))
print(f"Training loss: %.3f" % trainloss)

cv_scores = cross_val_score(model3, Xtrain3, ytrain, cv=kf, scoring=sc)
print(f"CV loss: %.3f +/- %.3f" % (cv_scores.mean(), cv_scores.std()))
Xtrain3.shape

Xtrain3 is of size  (310, 21)
Training loss: 30.709
CV loss: 35.333 +/- 4.774


(310, 21)

Now, which model to choose?

Model 1.

Why?

Because we choose the simplest model that is within one standard deviation of the model which has the lowest cross-validated training loss.

(36.009 is within 33.409 +/- 4.420)

## 3. Pipelines 

Pipelines allow us to "chain" different parts of a model building process. They are very useful to apply transformations: If the transformation is applied using the information on the test set, there is "test set leak".

It is always a best practice to create a pipeline in order to apply transformations to the data such as normalization. Here we'll use it for creating quadratic and cubic features. There is no test set leak risk here (why?) so this could be done more efficiently directly on the dataset, so this example is purely for illustrative purposes.

In [16]:
# Define the different pipelines 
model1 = Pipeline([
    ('linear_regression', LinearRegression())
])
model3 = Pipeline([
    ('poly', PolynomialFeatures(degree=2)),
    ('linear_regression', LinearRegression())
])

Pipelines accept any transformer you can think of, even allowing the use of [classes](https://docs.python.org/3/tutorial/classes.html), a general object. Here is an example.

In [17]:
# Model2 with custom transform (You can also use ColumnTransformer)
class Age2(BaseEstimator, TransformerMixin):  
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X.assign(age2 = X.age**2)
        return X

model2 = Pipeline([
    ('age2', Age2()),
    ('linear_regression', LinearRegression())
])

In [18]:
# Check training loss
print(f"CV Loss (Model 1): %.3f +/- %.3f" % (cross_val_score(model1, Xtrain, ytrain, cv=kf, scoring=sc).mean(),
                                            cross_val_score(model1, Xtrain, ytrain, cv=kf, scoring=sc).std())
     )
print(f"CV Loss (Model 2): %.3f +/- %.3f" % (cross_val_score(model2, Xtrain, ytrain, cv=kf, scoring=sc).mean(),
                                            cross_val_score(model2, Xtrain, ytrain, cv=kf, scoring=sc).std())
     )
print(f"CV Loss (Model 2): %.3f +/- %.3f" % (cross_val_score(model3, Xtrain, ytrain, cv=kf, scoring=sc).mean(),
                                            cross_val_score(model3, Xtrain, ytrain, cv=kf, scoring=sc).std())
     )

CV Loss (Model 1): 36.009 +/- 3.330
CV Loss (Model 2): 33.409 +/- 4.420
CV Loss (Model 2): 35.333 +/- 4.774


In [19]:
# Now report test loss on selected model. Always use an independent test for this!
model1 = model1.fit(Xtrain,ytrain)
testloss = mean_squared_error(ytest,model1.predict(Xtest))
print(f"Test loss: %.3f" % testloss)

Test loss: 35.636
